# SEAPORTS ARRIVALS DATA CLEANING
Additional information about measurement scale: 'Gross capacity' - tones, 'Length' - meters

In [1]:
import numpy as np 
import pandas as pd 

In [2]:
column_names = ['Destination seaport code', 'Arrival date', 'Arrival time', 'Departure seaport code', "Ship's name", 'Cockade', 'IMO number', 'Call sign', 'Gross capacity', 'Length', 'Agent', 'Berth']
darlowo = pd.read_csv('data\Dane_archiwalne_PRZYBYCIA_statków_Darłowo_2005_-_09.2020_DqzvqlF.csv', sep=';', names=column_names, encoding='latin2', skiprows=1)
gdansk = pd.read_csv('data\Dane_archiwalne_PRZYBYCIA_statków_do_portu_Gdańsk_2005-06.2020_aERLIVJ.csv', sep=';', names=column_names, encoding='latin2')
gdansk_nowy_port = pd.read_csv('data\Dane_archiwalne_PRZYBYCIA_statków_do_portu_Gdańsk_Nowy_Port_2005-06.2020.csv', sep=';', names=column_names, encoding='latin2', skiprows=1)
gdynia = pd.read_csv('data\Dane_archiwalne_PRZYBYCIA_statków_do_portu_Gdynia_2005-06.2020.csv', sep=';', names=column_names, encoding='latin2', skiprows=1)
police = pd.read_csv('data\Dane_archiwalne_PRZYBYCIA_statków_do_portu_Police_2008_-_09.2020_DpWrvve.csv', sep=';', names=column_names, encoding='latin2', skiprows=1)
swinoujscie = pd.read_csv('data\Dane_archiwalne_PRZYBYCIA_statków_do_portu_Świnoujście_2008_-_09.2020.csv', sep=';', names=column_names, encoding='latin2', skiprows=1)
szczecin = pd.read_csv('data\Dane_archiwalne_PRZYBYCIA_statków_do_portu_Szczecin_2008_-_09.2020_cmds6IL.csv', sep=';', names=column_names, encoding='latin2', skiprows=1)
ustka = pd.read_csv('data\Dane_archiwalne_PRZYBYCIA_statków_do_portu_Ustka_2005-06.2020.csv', sep=';', names=column_names, encoding='latin2', skiprows=1)
kolobrzeg = pd.read_csv('data\Dane_archiwalne_PRZYBYCIA_statków_Kołobrzeg_2005_-_09.2020.csv', sep=';', names=column_names, encoding='latin2', skiprows=1)

In [3]:
# create list of each seaport's data and its names
seaports = [darlowo, gdansk, gdansk_nowy_port, gdynia, police, swinoujscie, szczecin, ustka, kolobrzeg]
seaports_names = ['darlowo', 'gdansk', 'gdansk_nowy_port', 'gdynia', 'police', 'swinoujscie', 'szczecin', 'ustka', 'kolobrzeg']

## View dataframes shapes and cumulative number of rows

In [4]:
# view shape
for index, seaport in enumerate(seaports_names):
    print(seaport, ': ', seaports[index].shape)

# view cumulative number of rows
sum = 0
for seaport in seaports:
    sum += len(seaport)
print('Cumulative number of rows: ', sum)

darlowo :  (1100, 12)
gdansk :  (11557, 12)
gdansk_nowy_port :  (29720, 12)
gdynia :  (42729, 12)
police :  (3676, 12)
swinoujscie :  (65765, 12)
szczecin :  (38873, 12)
ustka :  (159, 12)
kolobrzeg :  (2703, 12)
Cumulative number of rows:  196282


## Merge data

In [5]:
df = pd.DataFrame()
for seaport in seaports:
    df = df.append(seaport)

## Remove unnecessary columns: 'Berth', 'Agent', "Ship's name", 'Call sign', 'Gross capacity'
### I have found out that ships names and call signs are used for identification, but they are likely to change. This makes IMO number the best for identification purpose, so I have removed those 2 columns.
### 'Berth', 'Agent' and 'Gross capacity' are useless for visualisation purpose.

In [6]:
df = df.drop(['Berth', 'Agent', "Ship's name", 'Call sign', 'Gross capacity'], axis=1)
df.columns

Index(['Destination seaport code', 'Arrival date', 'Arrival time',
       'Departure seaport code', 'Cockade', 'IMO number', 'Length'],
      dtype='object')

## View unique values in each column

In [7]:
for column in df.columns:
    print(column, ': ', df[column].nunique())

Destination seaport code :  9
Arrival date :  10447
Arrival time :  1206
Departure seaport code :  1626
Cockade :  186
IMO number :  11487
Length :  6858


## COLUMN: Arrival date
### Change 'Arrival date' column type and filter by years (we know that data about different seaports are from different periods, but we want them to be from the same)

In [8]:
# convert type
df['Arrival date'] = pd.to_datetime(df['Arrival date'])

In [9]:
start_date = '2008-01-01'
end_date = '2019-12-31'

# remove data from undesirable date range
df = df[(df['Arrival date'] >= start_date) & (df['Arrival date'] <= end_date)]
print('Arrival date range:  ', df['Arrival date'].min().date(), '  -  ', df['Arrival date'].max().date())

Arrival date range:   2008-01-01   -   2019-12-31


## COLUMN: Arrival day of week
### Adding column containing day of week of arrival

In [10]:
df['Arrival day of week'] = df['Arrival date'].dt.day_name()

## COLUMN: IMO number
### There is no accessible dataset with IMO numbers of ships, so I scrape this data (together with ship's length) from https://www.vesselfinder.com/

In [11]:
# check if IMO numbers follow official pattern (7-digits number)
df['IMO number'] = df['IMO number'].astype(str)
df['IMO number'][df['IMO number'].str.isnumeric() == False] = np.nan
df['IMO number'][df['IMO number'].str.len() != 7] = np.nan

# save IMO numbers with correct format as list
imo_numbers_list = df['IMO number'].unique().tolist()

### Use scraping script

In [12]:
from scraping_fun import scrape
scrape(imo_numbers_list)

In [13]:
# read data from file created by scrape()
scraped_length = pd.read_csv('data\scraped_length.csv')

# change type of imo number to be string (neccesary for further tasks)
scraped_length['imo'] = scraped_length['imo'].astype(pd.Int32Dtype()).astype(str)

# check if IMO number exists and placing np.nan for ships parameters if not
df['IMO number'][df['IMO number'].isin(scraped_length['imo']) == False] = np.nan
df['Cockade'][df['IMO number'].isin(scraped_length['imo']) == False] = np.nan
df['Length'][df['IMO number'].isin(scraped_length['imo']) == False] = np.nan

## COLUMN: Length

In [14]:
# create list of correct IMO numbers
correct_imo_numbers = df['IMO number'][df['IMO number'].isin(scraped_length["imo"]) == True].tolist()

# iterate through list of correct IMO numbers
for imo_number in correct_imo_numbers:
    # I use try and except to avoid error when trying to convert missing value to int
    try:
        # assign length from scraped data
        df['Length'][df['IMO number'] == imo_number] = int(scraped_length['length'][scraped_length['imo'] == imo_number])
    except:
        None

In [15]:
# convert numeric values to int to finally check their correctness
df['Length'] = df['Length'].astype(str).astype(float).round(0).astype(pd.Int32Dtype())
df['IMO number'] = df['IMO number'].astype(str).astype(float).round(0).astype(pd.Int32Dtype())

## COLUMN: Departure seaport code

In [16]:
# place NaNs when length if incorrect
df['Departure seaport code'] = df['Departure seaport code'].astype(str)
df['Departure seaport code'][df['Departure seaport code'].str.len() != 5] = np.nan

### Create dictionary with abbreviations and full names of countries

In [17]:
# read country abbreviations dataset
country_codes = pd.read_excel('data\-nzcs-344---nz-border-agencies-advance-notice-of-arrival---tsw-csv-upload-version.xlsx', sheet_name='Country Codes', header=4, usecols='B:C')

# view top 5 rows
print(country_codes.head())

# change type from dataframe to dictionary
country_codes = country_codes.set_index('Country Code').to_dict()['Name']

             Name Country Code
0     Afghanistan           AF
1   Aland Islands           AX
2         Albania           AL
3         Algeria           DZ
4  American Samoa           AS


### Create list of correct seaport codes

In [18]:
# read seaports information dataset
seaport_codes = pd.read_csv('data\code-list_csv.csv', header=0, usecols=[1,2])

# view top 5 rows
print(seaport_codes.head())

# combine country and location abbreviations to create seport code
seaport_codes['Seaport code'] = seaport_codes['Country'] + seaport_codes['Location']

# drop 'Country' and 'Location' columns
seaport_codes = seaport_codes.drop(['Country', 'Location'], axis=1)

# change list from column values
seaport_codes = seaport_codes['Seaport code'].tolist()

# view top 5 created seaport codes
seaport_codes[:5]

  Country Location
0      AD      ALV
1      AD      CAN
2      AD      ENC
3      AD      ESC
4      AD      EAC


['ADALV', 'ADCAN', 'ADENC', 'ADESC', 'ADEAC']

In [19]:
# place NaNs for incorrect seaport codes
df['Departure seaport code'][df['Departure seaport code'].isin(seaport_codes) == False] = np.nan

## ADD COLUMN: Departure country

In [20]:
# extract country abbreviation (2 first digits of seaport code) and replace using country_codes dictionary
df['Departure country'] = df['Departure seaport code'].str[0:2].replace(country_codes)

## COLUMN: Cockade

In [21]:
# view unique values of 'Cockade' column
df.Cockade = df.Cockade.astype(str)
print('Cackade column unique values: ', df.Cockade.nunique())
np.sort(df.Cockade.unique())

Cackade column unique values:  163


array(['AG', 'AN', 'ANTIGUA & BARBUDA', 'Afghanistan', 'Algeria',
       'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Azerbaijan', 'BB',
       'BE', 'BS', 'BZ', 'Bahamas', 'Bangladesh', 'Barbados', 'Belgium',
       'Belize', 'Bermuda', 'Brazil', 'British Indian Ocean Territory',
       'Bulgaria', 'COOK ISLANDS', 'CW', 'CY', 'Cambodia', 'Canada',
       'Cayman Islands', 'China', 'Cocos (Keeling) Islands', 'Comoros',
       'Cook Islands', 'Croatia', 'Curaçao', 'Cyprus', 'DE', 'DK',
       'Denmark', 'Dominica', 'Dominican Republic', 'Egypt', 'Estonia',
       'FI', 'FINLAND', 'FO', 'Faroe Islands', 'Finland', 'France',
       'French Polynesia', 'GB', 'GERMANY', 'GI', 'Georgia', 'Germany',
       'Ghana', 'Gibraltar', 'Greece', 'Greenland', 'HK', 'Honduras',
       'Hong Kong', 'Hong Kong ', 'Hungary', 'IE', 'IM', 'Iceland',
       'India', 'Indonesia', 'Ireland', 'Isle of Man', 'Israel', 'Italy',
       'Jamaica', 'Japan', 'KN', 'KY', 'Kiribati',
       "Korea Democratic People

In [22]:
%%capture
# correct and unify country names using country converter library
import country_converter as coco
df.Cockade = coco.convert(names=df.Cockade, to='name_short', not_found=None)

# replace 'nan' string with np.nan
df['Cockade'][df.Cockade == 'nan'] = np.nan

In [23]:
# view results
print('Cackade column unique values: ', df.Cockade.nunique())
np.sort(df.Cockade.astype(str).unique())

Cackade column unique values:  117


array(['AN', 'Afghanistan', 'Algeria', 'Anguilla', 'Antigua and Barbuda',
       'Argentina', 'Azerbaijan', 'Bahamas', 'Bangladesh', 'Barbados',
       'Belgium', 'Belize', 'Bermuda', 'Brazil',
       'British Indian Ocean Territory', 'Bulgaria', 'Cambodia', 'Canada',
       'Cayman Islands', 'China', 'Cocos (Keeling) Islands', 'Comoros',
       'Cook Islands', 'Croatia', 'Curacao', 'Cyprus', 'Denmark',
       'Dominica', 'Dominican Republic', 'Egypt', 'Estonia',
       'Faeroe Islands', 'Finland', 'France', 'French Polynesia',
       'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Greece', 'Greenland',
       'Honduras', 'Hong Kong', 'Hungary', 'Iceland', 'India',
       'Indonesia', 'Ireland', 'Isle of Man', 'Israel', 'Italy',
       'Jamaica', 'Japan', 'Kiribati', 'Kuwait', 'Latvia', 'Lebanon',
       'Liberia', 'Libya', 'Lithuania', 'Luxembourg', 'Macau', 'Malaysia',
       'Mali', 'Malta', 'Marshall Islands', 'Mauritius', 'Mexico',
       'Moldova', 'Montenegro', 'Myanmar', 'Netherla

## Delete empty rows

In [24]:
df = df.dropna(axis=0, how='all')

## Take last look at summary statistics

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182828 entries, 108 to 2630
Data columns (total 9 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   Destination seaport code  182828 non-null  object        
 1   Arrival date              182828 non-null  datetime64[ns]
 2   Arrival time              182828 non-null  object        
 3   Departure seaport code    170930 non-null  object        
 4   Cockade                   172031 non-null  object        
 5   IMO number                172031 non-null  Int32         
 6   Length                    172031 non-null  Int32         
 7   Arrival day of week       182828 non-null  object        
 8   Departure country         182828 non-null  object        
dtypes: Int32(2), datetime64[ns](1), object(6)
memory usage: 12.9+ MB


In [26]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
IMO number,172031.0,8.694822e+06,765601.705071,1002495.0,8124486.0,9019078.0,9255268.0,9869588.0
Length,172031.0,1.243571e+02,53.181478,10.0,87.0,125.0,164.0,400.0


In [27]:
df.head()

,Destination seaport code,Arrival date,Arrival time,Departure seaport code,Cockade,IMO number,Length,Arrival day of week,Departure country
108,PLDAR,2008-01-20,09:30:00,GBGSY,Russia,8700060,82,Sunday,United Kingdom
109,PLDAR,2008-11-19,09:40:00,DERSK,Antigua and Barbuda,6617855,68,Wednesday,Germany
110,PLDAR,2009-01-22,09:00:00,DKSKA,Panama,7013721,41,Thursday,Denmark
111,PLDAR,2009-01-25,08:00:00,DKSVE,St. Vincent and the Grenadines,7821049,63,Sunday,Denmark
112,PLDAR,2009-02-11,18:30:00,SEVAG,Germany,8975902,31,Wednesday,Sweden


## Save data to data.csv

In [28]:
df.to_csv('data\seaports_cleaned.csv', sep=',', index=False)